In [1]:
import dateutil
import pandas as pd

In [2]:
fr_raw = pd.read_csv(r'..\data\search\peak_finder\Final_Results.csv').dropna(axis=1, how='all')
fr_norm = pd.read_csv(r'..\data\processed\lipidomics_linregress_normalized_data.csv')
fr_filt = pd.read_csv(r'..\data\processed\lipidomics_normalized_filtered_data.csv', index_col=0)
file_grouping = pd.read_csv(r'..\data\metadata\file_grouping.csv', index_col=0)

fr_quant_filenames_in_analytical_order = file_grouping[file_grouping['quant_file'] == True]['fr_name'].to_list()

animals_used = [1091, 1093, 1060, 1062, 1074, 1092, 1102, 1076, 1082, 1101]
diabetic = [1076, 1082, 1101]
impaired = [1060, 1062, 1074, 1092, 1102]
normal = [1091, 1093]
bg_dict = {'diabetic': diabetic, 'impaired': impaired, 'normal': normal}
animal_to_tolerance_dict = {1076: 'diabetic', 1082: 'diabetic',1101: 'diabetic',
                            1060: 'impaired', 1062: 'impaired', 1074: 'impaired', 1092: 'impaired', 1102: 'impaired', 
                            1091: 'normal', 1093: 'normal',}

obs = pd.read_excel(r'..\data\metadata\20210421_FBGRBGOGTT_Huishi sample sheet.xlsx', sheet_name='cleaned')
obs['animal'] = obs['animal'].str.strip('TM').astype('int')
obs['datetime'] = obs['when'].apply(dateutil.parser.parse)
animal_phenotypes = pd.read_excel(r'..\data\metadata\20210421_FBGRBGOGTT_Huishi sample sheet.xlsx', sheet_name='animal phenotypes')
animal_phenotypes['animal'] = animal_phenotypes['animal'].str.strip('TM').astype('int')
metadata = pd.read_csv(r'..\data\metadata\20210923_huishi_metadata_scatterplot_request.csv', index_col=0)
metadata.index = metadata.index.str.split('M ').str[0].str.split('T').str[1]

In [5]:
groups = file_grouping[file_grouping['fr_name'].isin(fr_quant_filenames_in_analytical_order)] \
            [['fr_name', 'rat_label', 'bg_type', 'week', 'glucose_tolerance']]
groups['week'] = groups['week'].astype('int')
groups['rat_label'] = groups['rat_label'].astype('int')
groups = groups.set_index('fr_name')
groups = groups.T

df = fr_filt[fr_quant_filenames_in_analytical_order] # + ['Lipid Class']]
df = df.append(groups).T
df['week'] = df['week'].astype('int')
df['rat_label'] = df['rat_label'].astype('int')
df = df.set_index([df.index, 'rat_label', 'bg_type', 'week', 'glucose_tolerance']).T
feature_class = fr_filt['Lipid Class']
feature_class_mi = pd.MultiIndex.from_frame(feature_class.reset_index())
df = df.set_index(feature_class_mi)

df

,,20210729_AJ_Toh_RatBloodGlucose_T1076M_20210405_10wk_FBG.raw (F27),20210729_AJ_Toh_RatBloodGlucose_T1062M_20210322_8wk_FBG.raw (F9),20210729_AJ_Toh_RatBloodGlucose_T1093M_20210402_9wk_FBG.raw (F49),20210729_AJ_Toh_RatBloodGlucose_T1060M_20210325_8wk_RBG.raw (F4),20210729_AJ_Toh_RatBloodGlucose_T1091M_20210410_10wk_FBG.raw (F39),20210729_AJ_Toh_RatBloodGlucose_T1092M_20210406_9wk_RBG.raw (F44),20210729_AJ_Toh_RatBloodGlucose_T1093M_20210413_10wk_RBG.raw (F52),20210729_AJ_Toh_RatBloodGlucose_T1082M_20210405_10wk_FBG.raw (F33),20210729_AJ_Toh_RatBloodGlucose_T1060M_20210401_9wk_RBG.raw (F6),20210729_AJ_Toh_RatBloodGlucose_T1101M_20210402_9wk_FBG.raw (F55),...,20210729_AJ_Toh_RatBloodGlucose_T1092M_20210410_10wk_FBG.raw (F45),20210729_AJ_Toh_RatBloodGlucose_T1082M_20210325_8wk_RBG.raw (F30),20210729_AJ_Toh_RatBloodGlucose_T1062M_20210325_8wk_RBG.raw (F10),20210729_AJ_Toh_RatBloodGlucose_T1102M_20210410_10wk_FBG.raw (F63),20210729_AJ_Toh_RatBloodGlucose_T1093M_20210330_8wk_RBG.raw (F48),20210729_AJ_Toh_RatBloodGlucose_T1074M_20210329_9wk_FBG.raw (F19),20210729_AJ_Toh_RatBloodGlucose_T1062M_20210401_9wk_RBG.raw (F14),20210729_AJ_Toh_RatBloodGlucose_T1101M_20210330_8wk_RBG.raw (F54),20210729_AJ_Toh_RatBloodGlucose_T1092M_20210330_8wk_RBG.raw (F42),20210729_AJ_Toh_RatBloodGlucose_T1092M_20210413_10wk_RBG.raw (F46)
,rat_label,1076,1062,1093,1060,1091,1092,1093,1082,1060,1101,...,1092,1082,1062,1102,1093,1074,1062,1101,1092,1092
,bg_type,FBG,FBG,FBG,RBG,FBG,RBG,RBG,FBG,RBG,FBG,...,FBG,RBG,RBG,FBG,RBG,FBG,RBG,RBG,RBG,RBG
,week,10,8,9,8,10,9,10,10,9,9,...,10,8,8,10,8,9,9,8,8,10
,glucose_tolerance,diabetic,impaired,normal,impaired,normal,impaired,normal,diabetic,impaired,diabetic,...,impaired,diabetic,impaired,impaired,normal,impaired,impaired,diabetic,impaired,impaired
Unique Identifier,Lipid Class,,,,,,,,,,,,,,,,,,,,,
unknown_679.05463_0.536,unknown,1388749.128,10421872.865824,9761684.323293,16921439.53804,20688928.960266,15669494.721326,15473550.71764,10976959.20458,22473043.158375,11568964.50206,...,5778440.641962,10772995.12238,5531159.089789,9394719.924106,6994744.524707,4547156.482537,10709969.416478,12091064.707953,15211855.644512,12870363.786186
unknown_641.1087_0.537,unknown,3035805.818,25187896.703531,21759637.21643,53970365.787381,44964853.999588,32118819.369369,28807488.042992,27184415.156991,34856906.590804,32071862.575306,...,26580209.040746,13631037.452342,10933989.914237,21917324.579216,14501208.871636,4983605.791358,5986365.514152,24701156.987165,29588330.663132,25008307.122512
unknown_951.15973_0.541,unknown,1663566.711,13925677.102927,8616873.409517,31668716.622149,17280787.412394,18783610.036426,12963744.533145,17805969.390801,9434457.214292,16903858.872521,...,11480054.641035,6140714.458471,9971309.668317,13448834.403309,16656585.835021,12091643.69317,16807127.99205,15504703.058349,17600796.304189,11505247.236564
unknown_329.03067_0.542,unknown,22151942.96,117791789.303919,109682830.894935,66052943.57828,166812715.923247,169541225.239944,179026330.616622,157462193.916065,203255410.258634,146533218.048722,...,155319951.375407,76818576.539242,126198478.483618,117330026.998548,74209307.985385,104462043.706204,156045511.878862,131494549.000338,151554920.09114,93775353.834969
unknown_969.13147_0.544,unknown,681656.4071,6859485.144289,3460302.937599,16613341.086377,5240092.024078,9535689.552105,7278404.227445,7712597.657775,5525662.840195,6829185.243302,...,6367111.796283,3928397.339482,4907543.901523,5873009.985889,4845314.280741,4940117.900333,9316513.022112,7424280.702594,3528424.001508,4333286.595399


In [6]:
# Very useful!  https://stackoverflow.com/a/33298422

def filter_by(df, constraints, axis):
    """Filter MultiIndex by sublevels."""
    if axis == 0:
        indexer = [constraints[name] if name in constraints else slice(None)
                   for name in df.index.names]
        return df.loc[tuple(indexer)] if len(df.shape) == 1 else df.loc[tuple(indexer),]
    if axis == 1:
        indexer = [constraints[name] if name in constraints else slice(None)
                   for name in df.columns.names]
        return df.loc[:, tuple(indexer)] if len(df.shape) == 1 else df.loc[:, tuple(indexer),]        

pd.Series.filter_by = filter_by
pd.DataFrame.filter_by = filter_by

In [39]:
# Column levels:
# 1 = rat_label
# 2 = bg_type
# 3 = week
# 4 = glucose_tolerance
# 5 = bg 
# Can also access by level=string

df.filter_by({'rat_label': 1060,
              'bg_type': 'RBG'},
             axis=1)

,,20210729_AJ_Toh_RatBloodGlucose_T1060M_20210325_8wk_RBG.raw (F4),20210729_AJ_Toh_RatBloodGlucose_T1060M_20210401_9wk_RBG.raw (F6),20210729_AJ_Toh_RatBloodGlucose_T1060M_20210408_10wk_RBG.raw (F8)
,rat_label,1060,1060,1060
,bg_type,RBG,RBG,RBG
,week,8,9,10
,glucose_tolerance,impaired,impaired,impaired
Unique Identifier,Lipid Class,,,
unknown_679.05463_0.536,unknown,16921439.53804,22473043.158375,8914996.760126
unknown_641.1087_0.537,unknown,53970365.787381,34856906.590804,22502542.021439
unknown_951.15973_0.541,unknown,31668716.622149,9434457.214292,17472281.6349
unknown_329.03067_0.542,unknown,66052943.57828,203255410.258634,159882514.493045
unknown_969.13147_0.544,unknown,16613341.086377,5525662.840195,8111235.315606


In [11]:
df.filter_by({'Lipid Class': 'TG'},
             axis=0)

,,20210729_AJ_Toh_RatBloodGlucose_T1076M_20210405_10wk_FBG.raw (F27),20210729_AJ_Toh_RatBloodGlucose_T1062M_20210322_8wk_FBG.raw (F9),20210729_AJ_Toh_RatBloodGlucose_T1093M_20210402_9wk_FBG.raw (F49),20210729_AJ_Toh_RatBloodGlucose_T1060M_20210325_8wk_RBG.raw (F4),20210729_AJ_Toh_RatBloodGlucose_T1091M_20210410_10wk_FBG.raw (F39),20210729_AJ_Toh_RatBloodGlucose_T1092M_20210406_9wk_RBG.raw (F44),20210729_AJ_Toh_RatBloodGlucose_T1093M_20210413_10wk_RBG.raw (F52),20210729_AJ_Toh_RatBloodGlucose_T1082M_20210405_10wk_FBG.raw (F33),20210729_AJ_Toh_RatBloodGlucose_T1060M_20210401_9wk_RBG.raw (F6),20210729_AJ_Toh_RatBloodGlucose_T1101M_20210402_9wk_FBG.raw (F55),...,20210729_AJ_Toh_RatBloodGlucose_T1092M_20210410_10wk_FBG.raw (F45),20210729_AJ_Toh_RatBloodGlucose_T1082M_20210325_8wk_RBG.raw (F30),20210729_AJ_Toh_RatBloodGlucose_T1062M_20210325_8wk_RBG.raw (F10),20210729_AJ_Toh_RatBloodGlucose_T1102M_20210410_10wk_FBG.raw (F63),20210729_AJ_Toh_RatBloodGlucose_T1093M_20210330_8wk_RBG.raw (F48),20210729_AJ_Toh_RatBloodGlucose_T1074M_20210329_9wk_FBG.raw (F19),20210729_AJ_Toh_RatBloodGlucose_T1062M_20210401_9wk_RBG.raw (F14),20210729_AJ_Toh_RatBloodGlucose_T1101M_20210330_8wk_RBG.raw (F54),20210729_AJ_Toh_RatBloodGlucose_T1092M_20210330_8wk_RBG.raw (F42),20210729_AJ_Toh_RatBloodGlucose_T1092M_20210413_10wk_RBG.raw (F46)
,rat_label,1076,1062,1093,1060,1091,1092,1093,1082,1060,1101,...,1092,1082,1062,1102,1093,1074,1062,1101,1092,1092
,bg_type,FBG,FBG,FBG,RBG,FBG,RBG,RBG,FBG,RBG,FBG,...,FBG,RBG,RBG,FBG,RBG,FBG,RBG,RBG,RBG,RBG
,week,10,8,9,8,10,9,10,10,9,9,...,10,8,8,10,8,9,9,8,8,10
,glucose_tolerance,diabetic,impaired,normal,impaired,normal,impaired,normal,diabetic,impaired,diabetic,...,impaired,diabetic,impaired,impaired,normal,impaired,impaired,diabetic,impaired,impaired
Unique Identifier,Lipid Class,,,,,,,,,,,,,,,,,,,,,
TG_20:5_22:6_22:6_1014.755_13.093,TG,1798370.967,3823010.13815,3473810.07681,661777.445177,2216532.114415,29536.54748,118221.059575,3753589.884209,590372.608079,3603883.032147,...,2548058.881909,352744.729198,861864.641228,2319915.186325,59051.42594,1677008.381413,293283.815275,927789.561569,54662.082379,25681.444341
TG_22:6_22:6_22:6_1040.77014_13.389,TG,1600900.978,2238625.401863,996908.656694,598233.181944,841445.938555,13037.26149,75051.783414,1601585.608323,555937.754708,2460677.831093,...,863184.214952,637576.683639,847265.156907,1521042.797052,36373.777249,747186.485983,277096.530901,645499.514902,52318.458237,17333.203404
TG_18:2_20:5_20:5_945.69379_13.537,TG,4819765.07,6444548.186807,15543108.445011,3056149.165776,15913160.633463,741737.87318,2183878.606315,9669966.247181,2976323.15722,8107759.767752,...,12363038.603587,2488711.029071,2681265.648859,8764467.066629,830650.300288,7399181.315059,1878381.975594,5055292.909956,876616.409632,592291.69449
TG_60:13_966.75525_13.83,TG,19175571.47,45335326.401179,58637004.145456,19234738.97532,29040745.300246,2588337.848219,8534194.357662,35812844.119383,20887596.060259,52104734.94559,...,29114184.4881,13236217.209441,24271336.645507,51817082.032656,3173400.299914,21839550.626406,11150257.50585,25390270.599243,3048939.295292,1984913.60187
TG_18:2_22:6_22:6_997.72522_14.115,TG,15939686.56,13763092.893966,11129018.49998,7683312.240996,8821401.08058,800407.812017,2592902.38061,8258606.178998,8787415.486509,16389463.096527,...,7417882.931673,9452742.976137,8544108.197736,14349149.192925,1233522.60862,6983923.685334,4540458.726112,8097496.808089,1034019.452995,737876.09082


In [20]:
fr_filt.filter(regex='(?=.*1060)(?=.*RBG)', axis=1).mean(axis=1)

Unique Identifier
unknown_679.05463_0.536      1.610316e+07
unknown_641.1087_0.537       3.710994e+07
unknown_951.15973_0.541      1.952515e+07
unknown_329.03067_0.542      1.430636e+08
unknown_969.13147_0.544      1.008341e+07
                                 ...     
unknown_897.82532_20.148     2.380963e+05
unknown_974.9118_20.163      3.730609e+05
unknown_593.58716_20.175     1.773609e+05
unknown_871.80945_20.196     6.881140e+04
unknown_1003.93829_20.435    2.849300e+05
Length: 1029, dtype: float64

In [33]:
df.groupby(by='rat_label', axis=1)